# Lecture 19 Persuasion

This notebook will let you generate images using the OpenAI images API.

Below is the overview of this notebook.

<ol type = 1>
  <li> Generate targeted content based on user descriptions </li>
  <li> Generate targeted content based on user tweets </li>
  <li> Generate targeted video content based on user descriptions or tweets </li>
</ol>

This notebook can be opened in Colab
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zlisto/social_media_analytics/blob/main/Lecture19_Persuasion.ipynb)

Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab.  This will save the notebook in your Google Drive.


# Clones, Installs, and Imports

## Clone Repo

In [ ]:
# Clone GitHub repository
!git clone https://github.com/zlisto/social_media_analytics

import os
os.chdir("social_media_analytics")


## Installs

In [ ]:
# Install requirements
!pip install openai -q
!pip install umap-learn -q

## Imports

In [ ]:
import pandas as pd
import datetime
import textwrap as tr
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os
import time
import ast
import json
from IPython.display import Image,display, HTML, Audio
import random

from datetime import datetime, timedelta
import base64
import requests

import matplotlib.pyplot as plt
from PIL import Image as PILImage

import requests
from io import BytesIO
import numpy as np
import math

import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import requests
import os

import openai
import tqdm
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
from umap import UMAP
from sklearn.decomposition import PCA
import math


pd.set_option("display.max_colwidth", None)
plt.rcParams['figure.figsize'] = [8, 6]  # Width=8 inches, height=6 inches
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)

In [ ]:
#set your OpenAI API key as a string
OPENAI_API_KEY = ''

client = openai.Client(api_key=OPENAI_API_KEY)
#Pick your text model
#MODEL = "gpt-3.5-turbo-1106"
MODEL = "gpt-4-1106-preview"

## Helper Functions

In [ ]:
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_completion(prompt, instructions, client, model="gpt-3.5-turbo",
                   output_type = 'text'):
  '''Get a text completion from the OpenAI API'''
  completion = client.chat.completions.create(
                model=model,
                response_format={ "type": output_type},
                messages=[
                  {"role": "system", "content": instructions},
                  {"role": "user", "content": prompt}
                ]
              )
  response =completion.choices[0].message.content

  return response

def generate_image(prompt = "Draw a cute bunny", model = "dall-e-3"):
  '''Generates an image using the OpenAI API'''

  response_img = client.images.generate(
    model= model,
    prompt=prompt,
    size="1024x1024",
    quality="standard",
    n=1,
  )
  time.sleep(1)
  image_url = response_img.data[0].url
  revised_prompt = response_img.data[0].revised_prompt

  return image_url, revised_prompt

def generate_image_description(image_urls, instructions):
  '''Generates a description of a list of image_urls using the OpenAI Vision API'''
  PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [{"type": "text","text":instructions},
            *map(lambda x: {"type":"image_url","image_url": x}, image_urls),
        ],
    },
  ]
  params = {
      "model": "gpt-4-vision-preview",
      "messages": PROMPT_MESSAGES,
      "max_tokens": 1000,
  }

  response= client.chat.completions.create(**params)


  image_description = response.choices[0].message.content
  return image_description

def encode_image(image_path):
  '''Encodes an image to base64'''
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def display_image_url(image_url, width = 500, height = 500):
  '''Display the image located at image_url so it remains in the notebook
  even after the link dies '''
  response = requests.get(image_url)
  image_data = response.content
  # Encoding the image data as base64
  base64_image = base64.b64encode(image_data).decode('utf-8')
  # Generating HTML to display the image
  html_code = f'<img src="data:image/jpeg;base64,{base64_image}" width="{width}" height="{height}"/>'
  # Displaying the image in the notebook
  display(HTML(html_code))
  return html_code

def display_tweet(text='life is good', screen_name='zlisto'):
    display_html = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            .tweet {{
                background-color: white;
                color: black;
                border: 1px solid #e1e8ed;
                border-radius: 10px;
                padding: 20px;
                max-width: 500px;
                margin: 20px auto;
                font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
                box-shadow: 0px 0px 10px rgba(0,0,0,0.1);
            }}
            .user strong {{
                color: #1da1f2;
            }}
            .tweet-text p {{
                margin: 0;
                line-height: 1.5;
            }}
        </style>
    </head>
    <body>
        <div class="tweet">
            <div class="user">
                <strong>@{screen_name}</strong>
            </div>
            <div class="tweet-text">
                <p>{text}</p>
            </div>
        </div>
    </body>
    </html>
    '''
    display(HTML(display_html))
    return display_html

def display_IG(caption, image_url, screen_name='zlisto', profile_image_url = None):
    response = requests.get(image_url)
    image_data = response.content
    # Encoding the image data as base64
    base64_image = base64.b64encode(image_data).decode('utf-8')
    # Generating HTML to display the image
    image_url_local = f'data:image/jpeg;base64,{base64_image}'

    ''' HTML template for displaying the image, screen name, and caption in an Instagram-like format'''

    display_html = f"""
    <style>
        .instagram-post {{
            border: 1px solid #e1e1e1;
            border-radius: 3px;
            width: 600px;
            margin: 20px auto;
            background-color: white;
            font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
        }}
        .instagram-header {{
            padding: 14px;
            border-bottom: 1px solid #e1e1e1;
            display: flex;
            align-items: center;
        }}
        .instagram-profile-pic {{
            border-radius: 50%;
            width: 32px;
            height: 32px;
            margin-right: 10px;
        }}
        .instagram-screen-name {{
            font-weight: bold;
            color: #262626;
            text-decoration: none;
            font-size: 14px;
        }}
        .instagram-image {{
            max-width: 600px;
            width: auto;
            height: auto;
            display: block;
            margin: auto;
        }}
        .instagram-caption {{
            padding: 10px;
            font-size: 14px;
            color: #262626;
        }}
        .instagram-footer {{
            padding: 10px;
            border-top: 1px solid #e1e1e1;
        }}
        .instagram-likes {{
            font-weight: bold;
            margin-bottom: 8px;
        }}
    </style>
    <div class="instagram-post">
        <div class="instagram-header">
            <img src="{profile_image_url}" alt="Profile picture" class="instagram-profile-pic">
            <a href="#" class="instagram-screen-name">{screen_name}</a>
        </div>
        <img src="{image_url_local}" alt="Instagram image" class="instagram-image">
        <div class="instagram-caption">
            <a href="#" class="instagram-screen-name">{screen_name}</a> {caption}
        </div>
        <div class="instagram-footer">
            <div class="instagram-likes">24 likes</div>
            <!-- Include other footer content here -->
        </div>
    </div>
    """
    display(HTML(display_html))
    return display_html



# Targeted Content for Synthetic Users

We will create content that is designed to persuade sythethic users we create.

## Describe Synthetic Users

First we will describe the sythentic users we want to target.  Their descriptions are saved in a dataframe `df_targets`.

In [ ]:
targets = []

targets.append('''Tauhid, a professor at Yale, is aa young male who
has a PhD in computer science and
loves AI, crypto, Marvel movies, especially Avengers Endgame, and
hip hop, especially Jadakiss.''')

targets.append('''Alex, from Santa Fe, New Mexico, embodies the region's
cultural blend with shoulder-length, sun-lightened brown hair, often
accessorized with a turquoise necklace. They wear a mix of traditional
and modern eco-friendly clothing, reflecting Santa Fe's artistic and
environmental values. Professionally, Alex might be an artist or gallery
professional, deeply connected to the history and future of Southwestern
art, and enjoys hiking and attending local festivals in their free time.''')

targets.append('''Cheryl, a woman from Chicago. She’s got that urban,
Windy City flair—confident and walking with purpose. Her style is a
mix of high-street fashion with thrift shop finds; think a sleek blazer
paired with vintage jeans. She's likely got headphones on, nodding to
some Chance the Rapper, a local legend, as she navigates the crowded
sidewalks. There’s an energy about her that’s a blend of ambition and
a deep appreciation for the art and culture that’s etched into the
very architecture of the city. She’s a professional, working at one of
the city’s many startups, and after hours, you might find her at a
poetry slam, an improv comedy show, or enjoying a deep-dish pizza with
friends. Her vibe is very much Chicago: resilient, diverse, and ever-evolving.''')

df_targets = pd.DataFrame({'description':targets})
df_targets


## Define the Persuasion Topic

We choose the `topic` we want to persuade these users to support.

In [ ]:
topic = '''The government should provide student debt relief. '''

print(tr.fill(topic))


## Create Targeted Tweets

Choose your `content` to be a tweet.  Then use the `instructions` and the `description` of the user to create the content.

In [ ]:
content = 'tweet'

instructions = f'''You will be given a description of a target user.
Convince them to support {topic} with a {content}.
Dont directly mention the information about the target in the message.
Be more subtle as that makes the persuasion more effective. '''

for index,row in df_targets.iterrows():
  description = row['description']
  prompt = f'''User description:\n{description}.'''
  tweet = get_completion(prompt, instructions,client,MODEL)
  df_targets.loc[index,'tweet'] = tweet
  print(f"\n{tr.fill(description)}\n")
  display_tweet(tweet)




## Create Targeted Instagram Posts

Choose your `content` to be an Instagram image.  Then use the `instructions` and the `description` of the user to create the content (image and caption).

In [ ]:
content = "description of an image to be posted to Instagram."

instructions = f'''You will be given a description of a person.
Convince them to support {topic} with a {content}.
Dont directly mention the information about the target in the message.
Be more subtle as that makes the persuasion more effective. '''

for index, row in df_targets.iterrows():
  description = row['description']
  prompt = f'''User description:\n{description}.'''

  instructions_caption = f'''Write an Instagram caption for this image
  that will persuade a person with description {description} to
  support {topic} '''

  #Chat-GPT
  image_prompt = get_completion(prompt, instructions,client,MODEL)
  #DALL-E 3
  image_url, revised_prompt = generate_image(image_prompt)
  #Vision
  caption = generate_image_description(image_urls, instructions_caption)

  print(f"\n{tr.fill(description)}")
  print(f"\n{tr.fill(image_prompt)}")
  display_IG(caption, image_url)

  df_targets.loc[index,'image_prompt'] = image_prompt
  df_targets.loc[index,'image_url'] = image_url
  df_targets.loc[index,'caption'] = caption

# Geometry of Users, Topic, and Targeted Content

We can plot the embeddings of the topic, the user descriptions, and the targeted content to understand the geometry of the targeted content.

## Compute Embeddings of Topic, Targets, and Content



In [ ]:
df_targets['embedding_target'] = df_targets['description'].progress_apply(get_embedding)
df_targets['embedding_content'] = df_targets['tweet'].progress_apply(get_embedding)

v = get_embedding(topic)


## Compute Low-Dimensional Embeddings

In [ ]:
T = np.stack(df_targets['embedding_target'].values, axis = 0)
C = np.stack(df_targets['embedding_content'].values, axis = 0)
TC = np.vstack([T,C, [v]])


pca = PCA(n_components=2)
pca_embedding = pca.fit_transform(TC)

umap = UMAP(n_components=2)
umap_embedding = umap.fit_transform(TC)

ntargets = len(df_targets)
df_targets['target_pca_x'] = pca_embedding[0:ntargets,0]
df_targets['target_pca_y'] = pca_embedding[0:ntargets:,1]

df_targets['content_pca_x'] = pca_embedding[ntargets:2*ntargets,0]
df_targets['content_pca_y'] = pca_embedding[ntargets:2*ntargets,1]

df_targets['target_umap_x'] = umap_embedding[0:ntargets,0]
df_targets['target_umap_y'] = umap_embedding[0:ntargets:,1]

df_targets['content_umap_x'] = umap_embedding[ntargets:2*ntargets,0]
df_targets['content_umap_y'] = umap_embedding[ntargets:2*ntargets,1]

v_pca = pca_embedding[-1,:]

v_umap = umap_embedding[-1,:]

## Plot Low-Dimensional Embeddings

You should see the targeted content bridges the topic and the user description.  This is most clear in the UMAP embedding.

In [ ]:
embedding_type = 'umap'
s = 250

#plot the user description embedding
sns.scatterplot(x=f'target_{embedding_type}_x',
                y=f'target_{embedding_type}_y',
                data=df_targets, hue='description',
                marker = 'x',
                s = s)

#plot the content embedding
sns.scatterplot(x=f'content_{embedding_type}_x',
                y=f'content_{embedding_type}_y',
                data=df_targets, hue='description',
                marker = 'd',
                s = s)

#plot the topic embedding
xy = v_umap if embedding_type == 'umap' else v_pca

plt.scatter(xy[0], xy[1], color='black',
            marker = 'd', s=2*s, label = 'Topic')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()
plt.show()

# Targeting Content Based on User Tweets

We do not need to provide descriptions of our target users.  We can just provide the tweets they posted on social media.  The AI will figure out how to persuade them based on their tweets.

## Load a User's Tweets

In [27]:
df_tweets = pd.read_csv('data/tweets_TwExportly_sentiments.csv')
df_tweets.screen_name.unique()

screen_names = ['1future', 'elonmusk',
                'kamalaharris',
                'StephenCurry30']

## Generate Targeted Tweet Based on User Tweets

In [29]:
content = "tweet"

instructions = f'''You will be given a tweets posted by a person.
Create a {content} that will convence them to support: {topic}.
Dont directly mention the information about the target in the message.
Be more subtle as that makes the persuasion more effective. '''

nsamples = 100
for screen_name in screen_names:
  df = df_tweets[df_tweets.screen_name == screen_name].sample(nsamples).copy()
  tweets = "\n".join(df.text.tolist())
  prompt = f"Tweets of person: {tweets}"
  print(f"\n{screen_name}")
  tweet = get_completion(prompt, instructions, client, MODEL)
  display_tweet(tweet, screen_name);


## Generate Targeted Image Content Based on Tweets

In [31]:
content = "description of an image to be posted to Instagram."

instructions = f'''You will be given a tweets posted by a person.
Convince them to support {topic} with a {content}.
Dont directly mention the information about the target in the message.
Be more subtle as that makes the persuasion more effective. '''

nsamples = 100
for screen_name in screen_names:
  df = df_tweets[df_tweets.screen_name == screen_name].sample(nsamples).copy()
  tweets = "\n".join(df.text.tolist())
  prompt = f"Tweets of person: {tweets}"
  print(f"\n{screen_name}")
  image_prompt = get_completion(prompt, instructions,client,MODEL)
  image_url, revised_prompt = generate_image(image_prompt)

  instructions_caption = f'''Write an Instagram caption for this image
  that will persuade a person with these tweets:<{tweets}>,\n
  to support {topic}.  Return only the caption.'''

  caption = generate_image_description(image_urls, instructions_caption)


  print(f"\n{tr.fill(image_prompt)}")
  display_IG(caption, image_url, screen_name)





1future

elonmusk

kamalaharris

StephenCurry30


# Targeting Video Content

## Choose a Target

We have a few targets with descriptions in `df_targets` and we have tweets of users in `df_tweets`.  Choose someone from these datasets to be the target user.

In [32]:
TARGET = 'tweets'


#target from df_tweets
if TARGET == 'tweets':
  df_tweets = pd.read_csv('data/tweets_TwExportly_sentiments.csv')
  screen_name = '1future'
  nsamples = 100
  df = df_tweets[df_tweets.screen_name==screen_name].sample(nsamples)
  description = "".join(df.text.tolist())
  print(f"{screen_name}")

else:
  #target from df_targets
  description = df_targets.loc[0,'description']
  print(f"{tr.fill(description)}")



1future


## Write Script of Video

In [ ]:
instructions = f'''You will be given tweets posted by a target user.
You will convince this person to support {topic} by creating a TikTok video
with 3 scenes.
Write a script for this video and the voice narration for each scene.
Your script format should be a list of JSON objects
with keys "scene" (scene number) and "description" (description of scene),
and "narration" (narration of scene).'''


script = get_completion(description, instructions,client,MODEL)
df_script = pd.read_json(script.replace("json",'').replace("`",""))
df_script.head()

## Create Images for Each Scene

In [ ]:
for index, row in df_script.iterrows():
  scene = row['scene']
  description = row['description']
  narration = row['narration']
  image_url, revised_prompt = generate_image(description)
  print(image_url)
  display_IG(narration, image_url)
  df_script.loc[index,'image_url'] = image_url
